In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from transformers import LongformerTokenizer
filepath=[r"B:\project\human\human_chunk_1.csv",r"B:\project\ai\ai_chunk_1.csv"]
dfs = [pd.read_csv(filepath) for filepath in filepath]
df = pd.concat(dfs, ignore_index=True)
print(df.shape)

(30580, 2)


In [3]:
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
essay=df['text'].tolist()
tokens=tokenizer(essay,return_tensors='pt',padding=True,truncation=True)


In [4]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
xtrain_np=xtrain.numpy()
xtest_np=xtest.numpy()


In [5]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense

# Create a Sequential model
model = Sequential()

# Define the input length (assuming it's the same as before)
input_length = None

# Add the Embedding layer
model.add(Embedding(input_dim=52000, output_dim=128, input_length=input_length))

# Replace the LSTM layer with a GRU layer
model.add(GRU(64))

# Add the Dense output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(xtrain_np, ytrain, epochs=5, batch_size=64)



Epoch 1/5


383/383 [==============================] - 493s 1s/step - loss: 0.6935 - accuracy: 0.4988
Epoch 2/5
383/383 [==============================] - 504s 1s/step - loss: 0.6933 - accuracy: 0.5013
Epoch 3/5
383/383 [==============================] - 505s 1s/step - loss: 0.6933 - accuracy: 0.4999
Epoch 4/5
383/383 [==============================] - 504s 1s/step - loss: 0.6932 - accuracy: 0.5011
Epoch 5/5
383/383 [==============================] - 509s 1s/step - loss: 0.6932 - accuracy: 0.4972


In [ ]:
from keras.models import Model
from keras.layers import Input, Embedding, Conv1D, MaxPooling1D, GRU, concatenate, Flatten, Dense

# Define input shape
input_length = None
input_layer = Input(shape=(input_length,))

# Embedding layer
embedding_layer = Embedding(input_dim=52000, output_dim=128, input_length=input_length)(input_layer)

# CNN branch
cnn_layer = Conv1D(filters=64, kernel_size=5, activation='relu')(embedding_layer)
cnn_layer = MaxPooling1D(pool_size=2)(cnn_layer)

# GRU branch
gru_layer = GRU(64)(embedding_layer)

# Merge the outputs of the CNN and GRU branches
merged_layer = concatenate([cnn_layer, gru_layer])

# Flatten the merged output
flattened_layer = Flatten()(merged_layer)

# Dense layers
dense_layer = Dense(64, activation='relu')(flattened_layer)
output_layer = Dense(1, activation='sigmoid')(dense_layer)

# Create the combined model
combined_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
combined_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
combined_model.summary()
